In [2]:
# STEP 1: Imports
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pickle

# Suppress warnings
warnings.filterwarnings('ignore')

In [3]:
# STEP 2: Load and clean data
df = pd.read_csv('Earthquake_Data.csv', delimiter=r'\s+')

# Rename columns for better readability
new_column_names = ["Date(YYYY/MM/DD)", "Time(UTC)", "Latitude(deg)", "Longitude(deg)", "Depth(km)",
                    "Magnitude(ergs)", "Magnitude_type", "No_of_Stations", "Gap", "Close", "RMS",
                    "SRC", "EventID"]
df.columns = new_column_names

# Combine Date and Time columns into a datetime object and set as index
ts = pd.to_datetime(df["Date(YYYY/MM/DD)"] + " " + df["Time(UTC)"])
df = df.drop(["Date(YYYY/MM/DD)", "Time(UTC)"], axis=1)
df.index = ts

In [4]:
# STEP 2.1: EDA (Exploratory Data Analysis)
print("Dataset Info:")
print(df.info())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18030 entries, 1966-07-01 09:41:21.820000 to 2007-12-28 23:20:28.120000
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Latitude(deg)    18030 non-null  float64
 1   Longitude(deg)   18030 non-null  float64
 2   Depth(km)        18030 non-null  float64
 3   Magnitude(ergs)  18030 non-null  float64
 4   Magnitude_type   18030 non-null  object 
 5   No_of_Stations   18030 non-null  int64  
 6   Gap              18030 non-null  int64  
 7   Close            18030 non-null  int64  
 8   RMS              18030 non-null  float64
 9   SRC              18030 non-null  object 
 10  EventID          18030 non-null  int64  
dtypes: float64(5), int64(4), object(2)
memory usage: 1.7+ MB
None


In [19]:
print("Summary Statistics:")
print(df.describe())

Summary Statistics:
       Latitude(deg)  Longitude(deg)     Depth(km)  Magnitude(ergs)  \
count   18030.000000    18030.000000  18030.000000     18030.000000   
mean       37.639650     -120.921935      8.876301         3.427692   
std         1.921843        2.341440      7.698564         0.437849   
min        31.725700     -127.507000      0.000000         3.000000   
25%        36.463700     -122.004325      4.860000         3.110000   
50%        37.454500     -121.080700      7.070000         3.300000   
75%        38.829925     -118.862000     10.590000         3.600000   
max        45.562700     -112.107200    121.310000         7.390000   

       No_of_Stations           Gap         Close           RMS       EventID  
count    18030.000000  18030.000000  18030.000000  18030.000000  1.803000e+04  
mean        31.943150    147.555851     28.965225      0.142098  6.515898e+06  
std         24.535714     90.675337     42.751417      0.807726  1.233624e+07  
min          4.00000

In [20]:
print("Null Values:")
print(df.isnull().sum())

Null Values:
Latitude(deg)      0
Longitude(deg)     0
Depth(km)          0
Magnitude(ergs)    0
Magnitude_type     0
No_of_Stations     0
Gap                0
Close              0
RMS                0
SRC                0
EventID            0
dtype: int64


In [21]:
# Check unique values in categorical columns
if 'Magnitude_type' in df.columns:
    print("Magnitude Type Counts:")
    print(df['Magnitude_type'].value_counts())

if 'SRC' in df.columns:
    print("\nSource (SRC) Counts:")
    print(df['SRC'].value_counts())

Magnitude Type Counts:
Magnitude_type
Md    12353
ML     5499
Mw       97
Mx       81
Name: count, dtype: int64

Source (SRC) Counts:
SRC
NCSN    18030
Name: count, dtype: int64


In [22]:
# STEP 3: Encode categorical columns
df['Magnitude_type'] = LabelEncoder().fit_transform(df['Magnitude_type'])
df['SRC'] = LabelEncoder().fit_transform(df['SRC'])

In [23]:
# STEP 4: Feature Engineering

# 4.1: Time-based features
df['Hour'] = df.index.hour
df['Day'] = df.index.day
df['Month'] = df.index.month
df['Year'] = df.index.year

# 4.2: Magnitude Category
df['Magnitude_category'] = pd.cut(df['Magnitude(ergs)'], 
                                  bins=[0, 5.0, 6.0, 7.0, 10], 
                                  labels=['Low', 'Moderate', 'Strong', 'Severe'])
df['Magnitude_category'] = LabelEncoder().fit_transform(df['Magnitude_category'].astype(str))

# 4.3: Latitude and Longitude Zones
df['Lat_zone'] = pd.cut(df['Latitude(deg)'], bins=5, labels=False)
df['Lon_zone'] = pd.cut(df['Longitude(deg)'], bins=5, labels=False)

# 4.4: Depth Category
df['Depth_category'] = pd.cut(df['Depth(km)'], bins=[0, 70, 300, 700], labels=['Shallow', 'Intermediate', 'Deep'])
df['Depth_category'] = LabelEncoder().fit_transform(df['Depth_category'].astype(str))

# 4.5: Interaction Features
df['Station_Gap'] = df['No_of_Stations'] * df['Gap']
df['Depth_Close'] = df['Depth(km)'] / (df['Close'] + 1e-6)  # Avoid division by zero

In [24]:
# STEP 5: Feature Selection
X = df.drop(columns=['Magnitude(ergs)', 'EventID'])  # Drop target and ID columns
y = df['Magnitude(ergs)']  # Set target variable

In [25]:
# STEP 6: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

In [26]:
# STEP 7: Train Baseline Model (Linear Regression)
from sklearn.linear_model import LinearRegression

# Train a simple linear regression model as the baseline model
baseline_model = LinearRegression()

# Fit the baseline model
baseline_model.fit(X_train, y_train)

# Predict using the baseline model
y_pred_baseline = baseline_model.predict(X_test)

# Calculate metrics for the baseline model
mse_baseline = mean_squared_error(y_test, y_pred_baseline)
r2_baseline = r2_score(y_test, y_pred_baseline)
mae_baseline = mean_absolute_error(y_test, y_pred_baseline)

# Print metrics for the baseline model
print("Baseline Model Evaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse_baseline:.4f}")
print(f"R² Score: {r2_baseline:.4f}")
print(f"Mean Absolute Error (MAE): {mae_baseline:.4f}")


Baseline Model Evaluation Metrics:
Mean Squared Error (MSE): 0.1413
R² Score: 0.2660
Mean Absolute Error (MAE): 0.2883


In [30]:
# STEP 8: Hyperparameter Tuning with GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=0),
                           param_grid=param_grid,
                           scoring='r2',
                           cv=3,
                           n_jobs=-1,
                           verbose=1)

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_  # Best model from grid search

Fitting 3 folds for each of 16 candidates, totalling 48 fits


In [32]:
# STEP 9: Evaluate Model
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Print evaluation metrics
print("Random Forest Regressor Model Evaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")


Random Forest Regressor Model Evaluation Metrics:
Mean Squared Error (MSE): 0.1056
R² Score: 0.4511
Mean Absolute Error (MAE): 0.2445


In [33]:
# STEP 10: Save Model
with open('earthquake_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)
    print("✅ Model Saved Successfully")


✅ Model Saved Successfully


In [34]:
import pickle
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

In [35]:
# Load the model
with open('earthquake_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

print("Model loaded successfully.")

Model loaded successfully.


In [36]:
# Function to classify risk based on magnitude
def classify_risk(magnitude):
    if magnitude < 2.0:
        return "No Risk"
    elif 2.0 <= magnitude < 4.0:
        return "Very Low Risk"
    elif 4.0 <= magnitude < 5.0:
        return "Low Risk"
    elif 5.0 <= magnitude < 6.0:
        return "Moderate Risk"
    elif 6.0 <= magnitude < 7.0:
        return "High Risk"
    elif 7.0 <= magnitude < 8.0:
        return "Severe Risk"
    else:
        return "Extreme Risk"

In [38]:
# STEP C: Take input and predict using loaded model
print("\n--- Predict Earthquake Magnitude using Loaded Model ---")
n = int(input("How many predictions do you want to make? "))
new_data = []
label_encoder = LabelEncoder()

for i in range(n):
    print(f"\nEnter details for data point {i+1}:")
    latitude = float(input("Latitude(deg): "))
    longitude = float(input("Longitude(deg): "))
    depth = float(input("Depth(km): "))
    stations = int(input("No_of_Stations: "))
    gap = float(input("Gap: "))
    close = float(input("Close: "))
    rms = float(input("RMS: "))
    mag_type = int(input("Magnitude_type (as encoded number): "))
    src = int(input("SRC (as encoded number): "))
    
    # Enter DateTime (this will be used for Hour, Day, Month, Year)
    date_time_str = input("Enter DateTime (YYYY/MM/DD HH:MM:SS): ")
    date_time_obj = datetime.strptime(date_time_str, '%Y/%m/%d %H:%M:%S')
    
    # Extract Hour, Day, Month, Year
    hour = date_time_obj.hour
    day = date_time_obj.day
    month = date_time_obj.month
    year = date_time_obj.year
        # Feature engineering (replicating the transformations from the training data)
    lat_zone = pd.cut([latitude], bins=5, labels=False)[0]
    lon_zone = pd.cut([longitude], bins=5, labels=False)[0]
    
    # Magnitude category (you can map ranges directly as in the original code)
    magnitude_category = pd.cut([rms], bins=[0, 5.0, 6.0, 7.0, 10], labels=['Low', 'Moderate', 'Strong', 'Severe'])
    magnitude_category_encoded = label_encoder.fit_transform(magnitude_category.astype(str))[0]
    
    # Depth category (based on depth)
    depth_category = pd.cut([depth], bins=[0, 70, 300, 700], labels=['Shallow', 'Intermediate', 'Deep'])
    depth_category_encoded = label_encoder.fit_transform(depth_category.astype(str))[0]
    
    # Interaction features (same transformations from the training data)
    station_gap = stations * gap
    depth_close = depth / (close + 1e-6)  # Avoid division by zero
    
    # Creating a row of new input data for prediction
    new_data.append([latitude, longitude, depth, stations, gap, close, rms, mag_type, src, 
                    lat_zone, lon_zone, magnitude_category_encoded, depth_category_encoded, 
                    station_gap, depth_close, hour, day, month, year])



--- Predict Earthquake Magnitude using Loaded Model ---


How many predictions do you want to make?  1



Enter details for data point 1:


Latitude(deg):  23.45
Longitude(deg):  78.23
Depth(km):  10.0
No_of_Stations:  15
Gap:  45.0
Close:  12.3
RMS:  2.5
Magnitude_type (as encoded number):  0
SRC (as encoded number):  1
Enter DateTime (YYYY/MM/DD HH:MM:SS):  2023/05/18 13:45:00


In [39]:
# Convert new data into a DataFrame with the correct column order
new_data_df = pd.DataFrame(new_data, columns=[
    'Latitude(deg)', 'Longitude(deg)', 'Depth(km)', 'No_of_Stations', 'Gap', 'Close', 'RMS',
    'Magnitude_type', 'SRC', 'Lat_zone', 'Lon_zone', 'Magnitude_category', 'Depth_category',
    'Station_Gap', 'Depth_Close', 'Hour', 'Day', 'Month', 'Year'])

# Ensure the columns in the new data match the order of training data
training_columns = list(loaded_model.feature_names_in_)

# Reindex the new DataFrame to match the training column order
new_data_df = new_data_df.reindex(columns=training_columns)

In [40]:
# Make prediction with the model
predictions = loaded_model.predict(new_data_df)

print("Predictions using Loaded Model:")
for i, pred in enumerate(predictions):
    print(f"Prediction {i+1}: Magnitude(ergs) = {pred:.4f} \nThe location is at : {classify_risk(pred)}")

Predictions using Loaded Model:
Prediction 1: Magnitude(ergs) = 3.7343 
The location is at : Very Low Risk


In [41]:
import pandas as pd

df = pd.read_csv('Earthquake_Data.csv', delimiter=r'\s+', engine='python')
print(df)

      Date(YYYY/MM/DD)         Time  Latitude  Longitude  Depth   Mag Magt  \
0           1966/07/01  09:41:21.82   35.9463  -120.4700  12.26  3.20   Mx   
1           1966/07/02  12:08:34.25   35.7867  -120.3265   8.99  3.70   Mx   
2           1966/07/02  12:16:14.95   35.7928  -120.3353   9.88  3.40   Mx   
3           1966/07/02  12:25:06.12   35.7970  -120.3282   9.09  3.10   Mx   
4           1966/07/05  18:54:54.36   35.9223  -120.4585   7.86  3.10   Mx   
...                ...          ...       ...        ...    ...   ...  ...   
18025       2007/12/19  12:14:09.62   34.1438  -116.9822   7.03  4.06   ML   
18026       2007/12/21  12:14:56.45   37.3078  -121.6735   8.47  3.08   ML   
18027       2007/12/23  21:43:43.54   37.2127  -117.8230  10.00  3.54   ML   
18028       2007/12/28  01:59:42.40   36.5292  -121.1133   5.99  3.04   ML   
18029       2007/12/28  23:20:28.12   38.7710  -122.7370   2.34  3.40   Mw   

       Nst  Gap  Clo   RMS   SRC   EventID  
0        7  171   

In [42]:
import joblib

# Load the trained model correctly
model = joblib.load("earthquake_model.pkl")  # Make sure the file is in the same folder
print("model successfully loaded")
